In [2]:
#only run this if you restart the jupyter engine
#otherwise your paths will get messed up 

import requests
import sys 
import os
from pathlib import Path
os.chdir(Path("../"))

can_packet_parser = Path("../can_packet_parser")
server_path = Path("./server")
sys.path.append(str(can_packet_parser))
sys.path.append(str(server_path))
from server import model
from scrappy import yamls_to_packets, packet_t
from parse_can_log import *

In [3]:
packet_defs = yamls_to_packets("../../src/cangen/packets/")

In [6]:
#I save all the ASC can logs to this location.
#They can be found here: https://drive.google.com/file/d/1LbC-iea3hTGMYN80olSRcokneG2ogF-s/view?usp=share_link
#DO NOT PUSH THE CAN LOGS TO THE REPO
data_path = Path("data/ASC_CAN_LOGS/ASC_Day_3.csv")
day3 = load_data_from_telem_sd_log(data_path)

In [9]:
#This script will just spam the bus with as many packets as it can 
#caches to a list (similar to the actual implementation) 
#posts every 250 packets
from time import time_ns
import time
import math
post_avg = 0
bad_packets = 0
packet_cache = list()
for packet in day3:
    can_packet_t = packet_defs.get(packet[1], None)
    if can_packet_t is not None:
        parsed_packet = can_packet_t.parse_packet_to_json(packet[1], packet[2])

        for board in parsed_packet:
            for message in parsed_packet[board]:
                for idx in parsed_packet[board][message]:
                    packet_name: str
                    if can_packet_t.repeat > 1:
                        packet_name = f"{can_packet_t.name[packet[1]-can_packet_t.ids[0]]}_{can_packet_t.offset*(packet[1]-can_packet_t.ids[0])}"
                    else:
                        packet_name = f"{can_packet_t.name[0]}"  
                    p = {   "ID": idx,
                            "packet_name": packet_name,
                            "board": board,
                            "data": parsed_packet[board][message]
                    }
                    if packet_name != "bms_measurement":
                        continue
                    packet_cache.append(p)
                    #this is where you decide how often to post
                    if len(packet_cache) >= 50:
                        #records timing for each post
                        start_time = time_ns()
                        requests.post(f"http://127.0.0.1:5000/telemetry/{board}/{packet_name}", json = [p])
                        post_avg += (time_ns() - start_time)
                        packet_cache.clear()
    else: bad_packets += 1
post_avg /= (len(day3)-bad_packets)
print(f"Average Post time for {len(day3)-bad_packets} packets is: {post_avg} ns, {post_avg/math.pow(10, 6)}")

KeyboardInterrupt: 